In [ ]:
# %pip install langchain==0.3.20 | tail -n 1
# %pip install crewai==0.141.0 | tail -n 1
# %pip install langchain-community==0.3.19 | tail -n 1
# %pip install langchain-openai==0.3.25 | tail -n 1
# %pip install duckduckgo-search==7.5.2 | tail -n 1
# %pip install crewai-tools==0.51.1 | tail -n 1
# %pip install databricks-sdk==0.46.0 | tail -n 1

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.128 requires langchain-openai<0.3.0,>=0.2.1, but you have langchain-openai 0.3.25 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# !wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/3xGOgzMOv5jhRsA3A8N9fQ/leftover.py"

In [3]:
import sys
sys.path.append(".")

In [4]:
# from leftover import LeftoversCrew

In [5]:
# import os

# files = os.listdir('.')
# print(files)

In [6]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from IPython.display import display, JSON, Markdown
from datetime import datetime
import os

In [7]:
class GroceryItem(BaseModel):
    """Individual grocery item"""
    name: str = Field(description="Name of the grocery item")
    quantity: str = Field(description="Quantity needed (for example, '2 lbs', '1 gallon')")
    estimated_price: str = Field(description="Estimated price (for example, '$3-5')")
    category: str = Field(description="Store section (for example, 'Produce', 'Dairy')")

In [8]:
# sample_item = GroceryItem(
#     name="Chicken Breast",
#     quantity="2 lbs",
#     estimated_price="$8-12",
#     category="Meat"
# )

In [9]:
# sample_item

In [10]:
# type(sample_item)

In [11]:
# # Display structured data
# print("🛒 Sample Grocery Item Structure:")
# display(JSON(sample_item.model_dump()))

In [12]:
class MealPlan(BaseModel):
    """Simple meal plan"""
    meal_name: str = Field(description="Name of the meal")
    difficulty_level: str = Field(description="'Easy', 'Medium', 'Hard'")
    servings: int = Field(description="Number of people it serves")
    researched_ingredients: List[str] = Field(description="Ingredients found through research")

In [13]:
# sample_meal = MealPlan(
#     meal_name="Chicken Stir Fry",
#     difficulty_level="Easy",
#     servings=4,
#     researched_ingredients=["chicken breast", "broccoli", "bell peppers", "garlic", "soy sauce", "rice"]
# )

In [14]:
# sample_meal

In [15]:
# print("\n🍽️ Sample Meal Plan Structure:")
# display(JSON(sample_meal.model_dump()))

In [16]:
class ShoppingCategory(BaseModel):
    """Store section with items"""
    section_name: str = Field(description="Store section (for example, 'Produce', 'Dairy')")
    items: List[GroceryItem] = Field(description="Items in this section")
    estimated_total: str = Field(description="Estimated cost for this section")

In [17]:
# sample_section = ShoppingCategory(
#     section_name="Produce",
#     items=[
#         GroceryItem(name="Bell Peppers", quantity="3 pieces", estimated_price="$3-4", category="Produce"),
#         GroceryItem(name="Onions", quantity="2 lbs", estimated_price="$2-3", category="Produce")
#     ],
#     estimated_total="$5-7"
# )

In [18]:
# print("\n🏪 Sample Shopping Section:")
# display(JSON(sample_section.model_dump()))

In [19]:
class GroceryShoppingPlan(BaseModel):
    """Complete simplified shopping plan"""
    total_budget: str = Field(description="Total planned budget")
    meal_plans: List[MealPlan] = Field(description="Planned meals")
    shopping_sections: List[ShoppingCategory] = Field(description="Organized by store sections")
    shopping_tips: List[str] = Field(description="Money-saving and efficiency tips")

In [20]:
# weekly_shopping_plan = GroceryShoppingPlan(
#     total_budget="$40-50",
#     meal_plans=[breakfast_meal, lunch_meal, dinner_meal],  # List of MealPlan objects
#     shopping_sections=[dairy_section, meat_section, pantry_section, produce_section],  # List of ShoppingCategory objects
#     shopping_tips=[...]
# )

In [ ]:
%%capture

from crewai_tools import SerperDevTool
from crewai import Agent, Task, Crew, Process
from crewai import LLM 

# Initialize LLM
llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.7
)

In [ ]:
# Set up search tool (you'll need to add your API key)
os.environ['SERPER_API_KEY'] = '' 

In [ ]:
os.environ['GEMINI_API_KEY'] = '' 

In [24]:
meal_planner = Agent(
    role="Meal Planner & Recipe Researcher",
    goal="Search for optimal recipes and create detailed meal plans",
    backstory="A skilled meal planner who researches the best recipes online, considering dietary needs, cooking skill levels, and budget constraints.",
    tools=[SerperDevTool()],
    llm=llm,
    verbose=False
)

In [25]:
meal_planning_task = Task(
    description=(
        "Search for the best '{meal_name}' recipe for {servings} people within a {budget} budget. "
        "Consider dietary restrictions: {dietary_restrictions} and cooking skill level: {cooking_skill}. "
        "Find recipes that match the skill level and provide complete ingredient lists with quantities."
    ),
    expected_output="A detailed meal plan with researched ingredients, quantities, and cooking instructions appropriate for the skill level.",
    agent=meal_planner,
    output_pydantic=MealPlan,
    output_file="meals.json"
)

In [26]:
# from crewai import Crew, Process

# meal_planner_crew = Crew(
#     agents=[meal_planner],
#     tasks=[meal_planning_task],
#     process=Process.sequential,  # Ensures tasks are executed in order
#     verbose=True
# )

# meal_planner_result = meal_planner_crew.kickoff(
#     inputs={
#         "meal_name": "Chicken Stir Fry",
#         "servings": 4,
#         "budget": "$25",                           
#         "dietary_restrictions": ["no nuts"],       
#         "cooking_skill": "beginner"                
#     }
# )
# print("✅ Single meal planning completed!")
# print("📋 Single Meal Results:")
# print(meal_planner_result)

In [27]:
shopping_organizer = Agent(
    role="Shopping Organizer", 
    goal="Organize grocery lists by store sections efficiently",
    backstory="An experienced shopper who knows how to organize lists for quick store trips and considers dietary restrictions.",
    tools=[],
    llm=llm,
    verbose=False
)

In [28]:
shopping_task = Task(
    description=(
        "Organize the ingredients from the '{meal_name}' meal plan into a grocery shopping list. "
        "Group items by store sections and estimate quantities for {servings} people. "
        "Consider dietary restrictions: {dietary_restrictions} and cooking skill: {cooking_skill}. "
        "Stay within budget: {budget}."
    ),
    expected_output="An organized shopping list grouped by store sections with quantities and prices.",
    agent=shopping_organizer,
    context=[meal_planning_task],
    output_pydantic=GroceryShoppingPlan,
    output_file="shopping_list.json"
)

In [29]:
# two_agent_grocery_crew = Crew(
#     agents=[meal_planner, shopping_organizer],  # Both agents
#     tasks=[meal_planning_task, shopping_task],   # Both tasks
#     process=Process.sequential,
#     verbose=True
# )

# # Run the complete crew (this will do BOTH meal planning AND shopping)
# shopping_result = two_agent_grocery_crew.kickoff(
#     inputs={
#         "meal_name": "Chicken Stir Fry",
#         "servings": 4,
#         "budget": "$25",                           
#         "dietary_restrictions": ["no nuts"],      
#         "cooking_skill": "beginner"               
#     }
# )

# # Print the shopping results
# print("✅ Complete meal planning + shopping completed!")
# print("📋 Shopping Results:")
# print(shopping_result)

In [30]:
budget_advisor = Agent(
    role="Budget Advisor",
    goal="Provide cost estimates and money-saving tips",
    backstory="A budget-conscious shopper who helps families save money on groceries while respecting dietary needs.",
    tools=[SerperDevTool()],
    llm=llm,
    verbose=False
)

In [31]:
budget_task = Task(
    description=(
        "Analyze the shopping plan for '{meal_name}' serving {servings} people. "
        "Ensure total cost stays within {budget}. Consider dietary restrictions: {dietary_restrictions}. "
        "Provide practical money-saving tips and alternative ingredients if needed to meet budget."
    ),
    expected_output="A complete shopping guide with detailed prices, budget analysis, and money-saving tips.",
    agent=budget_advisor,
    context=[meal_planning_task, shopping_task],
    output_file="shopping_guide.md"
)

In [32]:
# !wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/-5NXnHGkX1WG81tqE8kSTg/agents.yaml"

In [33]:
# !wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7fhq1KPxi7cuqjOCBgcmXQ/tasks.yaml"

In [34]:
# %mkdir -p config
# %mv agents.yaml  config/agents.yaml
# %mv tasks.yaml   config/tasks.yaml

In [35]:
from leftover import LeftoversCrew

leftovers_cb = LeftoversCrew(llm=llm)
yaml_leftover_manager = leftovers_cb.leftover_manager()
yaml_leftover_task    = leftovers_cb.leftover_task()

In [36]:
summary_agent = Agent(
    role="Report Compiler",
    goal="Compile comprehensive meal planning reports from all team outputs",
    backstory="A skilled coordinator who organizes information from multiple specialists into comprehensive, easy-to-follow reports.",
    tools=[],
    llm=llm,
    verbose=False
)

In [37]:
summary_task = Task(
    description=(
        "Compile a comprehensive meal planning report that includes:\n"
        "1. The complete recipe and cooking instructions from the meal planner\n"
        "2. The organized shopping list with prices from the shopping organizer\n"
        "3. The budget analysis and money-saving tips from the budget advisor\n"
        "4. The leftover management suggestions from the waste reduction specialist\n"
        "Format this as a complete, user-friendly meal planning guide."
    ),
    expected_output="A comprehensive meal planning guide that combines all team outputs into one cohesive report.",
    agent=summary_agent,
    context=[meal_planning_task, shopping_task, budget_task, yaml_leftover_task],
)

In [38]:
summary_task = Task(
    description=(
        "Compile a comprehensive meal planning report that includes:\n"
        "1. The complete recipe and cooking instructions from the meal planner\n"
        "2. The organized shopping list with prices from the shopping organizer\n"
        "3. The budget analysis and money-saving tips from the budget advisor\n"
        "4. The leftover management suggestions from the waste reduction specialist\n"
        "Format this as a complete, user-friendly meal planning guide."
    ),
    expected_output="A comprehensive meal planning guide that combines all team outputs into one cohesive report.",
    agent=summary_agent,
    context=[meal_planning_task, shopping_task, budget_task, yaml_leftover_task],
)

In [39]:
complete_grocery_crew = Crew(
    agents=[
        meal_planner,           
        shopping_organizer,      
        budget_advisor,         
        yaml_leftover_manager,  
        summary_agent          
    ],
    tasks=[
        meal_planning_task,     
        shopping_task,          
        budget_task,            
        yaml_leftover_task,    
        summary_task            
    ],
    process=Process.sequential,
    verbose=True
)

In [40]:
# Run the complete crew
complete_result = complete_grocery_crew.kickoff(
    inputs={
        "meal_name": "Paneer Butter Masala",
        "servings": 4,
        "budget": "$25",
        "dietary_restrictions": [],
        "cooking_skill": "advance"
    }
)

print("✅ Complete meal planning with summary completed!")
print("📋 Complete Results:")
print(complete_result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3263955f-cef7-4141-a45f-5f951a32f0e7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Task: Search for the best 'Paneer Butter Masala' recipe for 4 people within a $25 budget. Consider dietary     │
│  restrictions: [] and cooking skill level: advance. Find recipes that match the skill level and provide         │
│  complete ingredient lists with quantities.                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Thought: I'll help you find the perfect Paneer Butter Masala recipe that meets the specified criteria. I'll    │
│  start by searching for advanced-level recipes.                                                                 │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"best authentic Paneer Butter Masala recipe advanced level professional chef\"}"          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Compiler                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  🍽️ COMPREHENSIVE MEAL PLANNING GUIDE: PANEER BUTTER MASALA 🍽️                                                    │
│                                                                                                                 │
│  I. RECIPE OVERVIEW                                                                                             │
│  - Dish: Paneer Butter Masala                                                                                   │
│  - Difficulty Level: Advanced                                                                                   │
│  - Servings: 4                                                                                                  │
│  - Total Preparation Time: Approximately 45-60 minutes                                                          │
│                                                                                                                 │
│  II. INGREDIENTS LIST                                                                                           │
│  Dairy & Refrigerated:                                                                                          │
│  - 200g paneer, cut into cubes                                                                                  │
│  - 1/2 cup heavy cream                                                                                          │
│  - 3 tablespoons unsalted butter                                                                                │
│                                                                                                                 │
│  Produce:                                                                                                       │
│  - 3 large ripe tomatoes, pureed                                                                                │
│  - 2 medium onions, finely chopped                                                                              │
│  - 1 inch fresh ginger, minced                                                                                  │
│  - 4-5 garlic cloves, crushed                                                                                   │
│  - Fresh cilantro for garnish                                                                                   │
│                                                                                                                 │
│  Spices & Dry Goods:                                                                                            │
│  - 10-12 whole cashew nuts                                                                                      │
│  - 2 tablespoons vegetable oil                                                                                  │
│  - 1 teaspoon cumin seeds                                                                                       │
│  - 1/2 tablespoon coriander powder                                                                              │
│  - 1 teaspoon garam masala                                                                                      │
│  - 1/4 teaspoon red chili powder                                                                                │
│  - Kasuri methi (dried fenugreek leaves)                                                                        │
│  - Salt to taste                                     

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e3d717ed-e94e-47e4-91b8-a19e806ed58e                                                                     │
│  Agent: Report Compiler                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3263955f-cef7-4141-a45f-5f951a32f0e7                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: 🍽️ COMPREHENSIVE MEAL PLANNING GUIDE: PANEER BUTTER MASALA 🍽️                                      │
│                                                                                                                 │
│  I. RECIPE OVERVIEW                                                                                             │
│  - Dish: Paneer Butter Masala                                                                                   │
│  - Difficulty Level: Advanced                                                                                   │
│  - Servings: 4                                                                                                  │
│  - Total Preparation Time: Approximately 45-60 minutes                                                          │
│                                                                                                                 │
│  II. INGREDIENTS LIST                                                                                           │
│  Dairy & Refrigerated:                                                                                          │
│  - 200g paneer, cut into cubes                                                                                  │
│  - 1/2 cup heavy cream                                                                                          │
│  - 3 tablespoons unsalted butter                                                                                │
│                                                                                                                 │
│  Produce:                                                                                                       │
│  - 3 large ripe tomatoes, pureed                                                                                │
│  - 2 medium onions, finely chopped                                                                              │
│  - 1 inch fresh ginger, minced                                                                                  │
│  - 4-5 garlic cloves, crushed                                                                                   │
│  - Fresh cilantro for garnish                                                                                   │
│                                                                                                                 │
│  Spices & Dry Goods:                                                                                            │
│  - 10-12 whole cashew nuts                                                                                      │
│  - 2 tablespoons vegetable oil                                                                                  │
│  - 1 teaspoon cumin seeds                                                                                       │
│  - 1/2 tablespoon coriander powder                                                                              │
│  - 1 teaspoon garam masala                                                                                      │
│  - 1/4 teaspoon red chili powder                                                                                │
│  - Kasuri methi (dried fenugreek leaves)            

✅ Complete meal planning with summary completed!
📋 Complete Results:
🍽️ COMPREHENSIVE MEAL PLANNING GUIDE: PANEER BUTTER MASALA 🍽️

I. RECIPE OVERVIEW
- Dish: Paneer Butter Masala
- Difficulty Level: Advanced
- Servings: 4
- Total Preparation Time: Approximately 45-60 minutes

II. INGREDIENTS LIST
Dairy & Refrigerated:
- 200g paneer, cut into cubes
- 1/2 cup heavy cream
- 3 tablespoons unsalted butter

Produce:
- 3 large ripe tomatoes, pureed
- 2 medium onions, finely chopped
- 1 inch fresh ginger, minced
- 4-5 garlic cloves, crushed
- Fresh cilantro for garnish

Spices & Dry Goods:
- 10-12 whole cashew nuts
- 2 tablespoons vegetable oil
- 1 teaspoon cumin seeds
- 1/2 tablespoon coriander powder
- 1 teaspoon garam masala
- 1/4 teaspoon red chili powder
- Kasuri methi (dried fenugreek leaves)
- Salt to taste

III. DETAILED COOKING INSTRUCTIONS
1. Cashew Nut Preparation:
- Soak cashew nuts in warm water for 15-20 minutes
- Drain and grind into a smooth paste

2. Tomato Base:
- Heat oil i